In [1]:
from pathlib import Path
import json
import os
from pprint import pprint
from rdflib import RDF, Graph, Namespace
from collections import defaultdict
from typing import List, Set, Dict
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Template
from buildingmotif.template_finder.template_finder import calculate_best_bindings_for_template_and_tokens
from buildingmotif.template_finder.classes import LabelSet, SegmentedLabel, Param, Token, BestLabelSetBindings

from datetime import datetime as dt
from dataclasses import dataclass

BLDG = Namespace("urn:building/")
PROJECT_DIR = Path(os.getcwd()).resolve().parent

In [2]:
bm = BuildingMOTIF("sqlite://")

Library.load(ontology_graph="https://github.com/BrickSchema/Brick/releases/download/nightly/Brick.ttl")
ground_truth = Library.load(ontology_graph=str(PROJECT_DIR / "gabes_points" / "sprucehall.ttl"))
equiment_templates = Library.load(directory=str(PROJECT_DIR / "gabes_points" / "equiment_templates"))

templates = equiment_templates.get_templates()


2023-10-04 13:27:08,332 | root |  WARNING: Warning: could not find dependee https://brickschema.org/schema/Brick/ref#ifcProject
2023-10-04 13:27:08,332 | root |  WARNING: Warning: could not find dependee n547a01f730484919a69060941df2212cb7045
2023-10-04 13:27:08,333 | root |  WARNING: Warning: could not find dependee n547a01f730484919a69060941df2212cb7047
2023-10-04 13:27:08,333 | root |  WARNING: Warning: could not find dependee n547a01f730484919a69060941df2212cb7049
2023-10-04 13:27:08,333 | root |  WARNING: Warning: could not find dependee n547a01f730484919a69060941df2212cb7051
2023-10-04 13:27:08,333 | root |  WARNING: Warning: could not find dependee n547a01f730484919a69060941df2212cb7053
2023-10-04 13:27:08,334 | root |  WARNING: Warning: could not find dependee n547a01f730484919a69060941df2212cb6972
2023-10-04 13:27:08,334 | root |  WARNING: Warning: could not find dependee n547a01f730484919a69060941df2212cb6974
2023-10-04 13:27:08,334 | root |  WARNING: Warning: could not find 

In [3]:
with open(PROJECT_DIR / "gabes_points" / "tokens.json") as f:
    labels_and_tokens = json.load(f)
    

In [4]:
# LabelSets are a set SegmentedLabels with shared token_classes. This cell builds that. 
labelsets: List[LabelSet] = []

for label_and_tokens in labels_and_tokens:
    label = label_and_tokens["label"]
    tokens = sorted(label_and_tokens["tokens"], key=lambda x: x["type"])
    token_classes = [t['type'] for t in tokens]
    token_identifiers = [t['identifier'] for t in tokens]

    # SegmentedLabel for label
    segmented_label = SegmentedLabel(
        label=label,
        identifiers=token_identifiers
    )

    # proper labelset for SegmentedLabel
    labelset = next((
        labelset for labelset in labelsets 
        if labelset.token_classes == token_classes
    ), None)
    if labelset is None:
        labelset = LabelSet(token_classes=token_classes, segmented_labels=[])
        labelsets.append(labelset)

    # append it to the label set
    labelset.segmented_labels.append(segmented_label)


# print
for labelset in labelsets:
    print(f"{len(labelset.segmented_labels)} labels with token classes:")
    pprint(labelset.token_classes)


273 labels with token classes:
['https://brickschema.org/schema/Brick#Air_Handling_Unit',
 'https://brickschema.org/schema/Brick#Chilled_Water_Valve',
 'https://brickschema.org/schema/Brick#Entity',
 'https://brickschema.org/schema/Brick#Fan_Coil_Unit']
1373 labels with token classes:
['https://brickschema.org/schema/Brick#Air_Handling_Unit',
 'https://brickschema.org/schema/Brick#Entity',
 'https://brickschema.org/schema/Brick#Entity',
 'https://brickschema.org/schema/Brick#Fan_Coil_Unit']
259 labels with token classes:
['https://brickschema.org/schema/Brick#Air_Handling_Unit',
 'https://brickschema.org/schema/Brick#Entity',
 'https://brickschema.org/schema/Brick#Fan_Coil_Unit',
 'https://brickschema.org/schema/Brick#Occupancy_Status']
259 labels with token classes:
['https://brickschema.org/schema/Brick#Air_Handling_Unit',
 'https://brickschema.org/schema/Brick#Entity',
 'https://brickschema.org/schema/Brick#Fan_Coil_Unit',
 'https://brickschema.org/schema/Brick#Unoccupied_Heating_Te

In [5]:
# get the best bindings for each labelset. 
# The binding's param's identifiers are the index of the labelset.segmented_labels[].identifiers
# so that the bindings can be applied to every labelset.segmented_labels
best_labelset_bindings_list = []
for labelset in labelsets:
    print(f"token classes: {labelset.token_classes}")
    print(labelset.segmented_labels[0])
    print()

    index_tokens = [
        Token(identifier=i, classname=tc) 
        for i, tc in enumerate(labelset.token_classes)
    ]
    best_template, best_bindings, _ = calculate_best_bindings_for_template_and_tokens(templates, index_tokens, verbose=True)
    best_labelset_bindings_list.append(BestLabelSetBindings(
        template=best_template,
        labelset=labelset,
        bindings_indices=best_bindings
    ))

    print(f"\nBest Template: {best_template.name}")
    print(f"bindings_indices: {best_bindings}")
    print("\n-----------------------------")

token classes: ['https://brickschema.org/schema/Brick#Air_Handling_Unit', 'https://brickschema.org/schema/Brick#Chilled_Water_Valve', 'https://brickschema.org/schema/Brick#Entity', 'https://brickschema.org/schema/Brick#Fan_Coil_Unit']
SegmentedLabel(label=':BuildingName_02:FCU503_ChwVlvPos', identifiers=['02', ':BuildingName_02:FCU503_ChwVlvPos', 'BuildingName', '503'])

Template costs:
cost matrix:
                              Air_Handling_Unit Chilled_Water_Valve Entity  \
Supply_Air_Temperature_Sensor               inf                 inf    inf   
Fan_Coil_Unit                               inf                 inf    inf   

                              Fan_Coil_Unit  
Supply_Air_Temperature_Sensor           inf  
Fan_Coil_Unit                             0  

kept edges:
Fan_Coil_Unit <- 0.0 -> Fan_Coil_Unit
- SaTmpEntity 3.0
cost matrix:
                                        Air_Handling_Unit Chilled_Water_Valve  \
Fan_Coil_Unit                                         inf    

In [6]:
# For each labelset, get the actualized best bindings. 
bindings_by_template_name = defaultdict(list)

for best_labelset_bindings in best_labelset_bindings_list:
    template_name = best_labelset_bindings.template.name
    bindings_list = best_labelset_bindings.get_bindings_list()

    print(f"Binding for {template_name} with token_classes {best_labelset_bindings.labelset.token_classes}")
    for bindings in bindings_list:
        print(f"   - {bindings.segmented_label.label}: {bindings.bindings}")
    
    bindings_by_template_name[template_name] += bindings_list

Binding for fcu with token_classes ['https://brickschema.org/schema/Brick#Air_Handling_Unit', 'https://brickschema.org/schema/Brick#Chilled_Water_Valve', 'https://brickschema.org/schema/Brick#Entity', 'https://brickschema.org/schema/Brick#Fan_Coil_Unit']
   - :BuildingName_02:FCU503_ChwVlvPos: {name a Fan_Coil_Unit: 503 a Fan_Coil_Unit, chw_coil-chw_vlv a Chilled_Water_Valve: :BuildingName_02:FCU503_ChwVlvPos a Chilled_Water_Valve}
   - :BuildingName_01:FCU362_ChwVlvPos: {name a Fan_Coil_Unit: 362 a Fan_Coil_Unit, chw_coil-chw_vlv a Chilled_Water_Valve: :BuildingName_01:FCU362_ChwVlvPos a Chilled_Water_Valve}
   - :BuildingName_01:FCU343_ChwVlvPos: {name a Fan_Coil_Unit: 343 a Fan_Coil_Unit, chw_coil-chw_vlv a Chilled_Water_Valve: :BuildingName_01:FCU343_ChwVlvPos a Chilled_Water_Valve}
   - :BuildingName_01:FCU301_ChwVlvPos: {name a Fan_Coil_Unit: 301 a Fan_Coil_Unit, chw_coil-chw_vlv a Chilled_Water_Valve: :BuildingName_01:FCU301_ChwVlvPos a Chilled_Water_Valve}
   - :BuildingName_02

In [25]:
# group the bindings by template, then by the value of the name. 
bindings_by_name_by_template_name = {}
for template_name, bindings_list in bindings_by_template_name.items():
    bindings_by_name = defaultdict(list)
    
    for bindings in bindings_list:
        name = next((
            token.identifier 
            for param, token in bindings.bindings.items()
            if param.name == "name"
        ), None)
        
        bindings_by_name[name].append(bindings)

    bindings_by_name_by_template_name[template_name] = bindings_by_name

for template_name, bindings_by_name in bindings_by_name_by_template_name.items():
    for name, bindings in bindings_by_name.items():
        print(f"{len(bindings)} bindings for {template_name} where name is {name}")
        unified_bindings = {}
        for b in bindings:
            unified_bindings.update(b.bindings)
        print(bindings[0].template)
        args = {p.name: BLDG[t.identifier] for p, t in unified_bindings.items()}
        templ = bindings[0].template.evaluate(args)
        if isinstance(templ, Template):
            _, graph = templ.fill(BLDG)
        else:
            graph = templ
        print(graph.serialize())
        break

9 bindings for fcu where name is 503
Template(_id=-1, _name='fcu', body=<Graph identifier=N29c31f6dcdaa454dae9e93776b4d2ab4 (<class 'rdflib.graph.Graph'>)>, optional_args=['unocc_htg_sp', 'occ_clg_sp', 'occ_htg_sp', 'occ_cmd', 'unocc_clg_sp', 'occ_status'], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x128a99850>)
@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:building/503> a brick:Fan_Coil_Unit ;
    brick:feeds <urn:building/zone_cebd7c35> ;
    brick:hasPart <urn:building/chw_coil_a6f56e14>,
        <urn:building/hw_coil_6c6c05ae> ;
    brick:hasPoint <urn:building/:BuildingName_02:FCU503_EffOcc>,
        <urn:building/:BuildingName_02:FCU503_OccClgSpt>,
        <urn:building/:BuildingName_02:FCU503_OccCmd>,
        <urn:building/:BuildingName_02:FCU503_OccHtgSpt>,
        <urn:building/:BuildingName_02:FCU503_UnoccClgSpt>,
        <urn:building/:BuildingName_02:FCU503_UnoccHtgSpt>,
        <urn:building/supply_temp_15396747> .

<urn:buil

In [8]:
# example: labels with best bindings on SaTmpEntity where name is "201"
for bindings in bindings_by_name_by_template_name["SaTmpEntity"]["251"]:
    print(bindings.bindings)

{name a Fan_Coil_Unit: 251 a Fan_Coil_Unit}
{name a Fan_Coil_Unit: 251 a Fan_Coil_Unit}
{name a Fan_Coil_Unit: 251 a Fan_Coil_Unit}
{name a Fan_Coil_Unit: 251 a Fan_Coil_Unit}
{name a Fan_Coil_Unit: 251 a Fan_Coil_Unit}
{name a Fan_Coil_Unit: 251 a Fan_Coil_Unit}
{name a Fan_Coil_Unit: 251 a Fan_Coil_Unit}
{name a Fan_Coil_Unit: 251 a Fan_Coil_Unit}
{name a Fan_Coil_Unit: 251 a Fan_Coil_Unit}
{name a Fan_Coil_Unit: 251 a Fan_Coil_Unit}
{name a Fan_Coil_Unit: 251 a Fan_Coil_Unit}
{name a Fan_Coil_Unit: 251 a Fan_Coil_Unit}
{sat a Supply_Air_Temperature_Sensor: :BuildingName_01:FCU251_UI22_SaTmp a Supply_Air_Temperature_Sensor, name a Fan_Coil_Unit: 251 a Fan_Coil_Unit}
